## Launch and set up a VM instance- with python-chi

We will use the `python-chi` Python API to Chameleon to provision our VM server.

We will execute the cells in this notebook inside the Chameleon Jupyter environment.

Run the following cell, and make sure the correct project is selected.

In [1]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

We will bring up a `m1.medium` flavor VM instance with the `CC-Ubuntu24.04` disk image.

> **Note**: the following cell brings up a server only if you don’t already have one with the same name! (Regardless of its error state.) If you have a server in ERROR state already, delete it first in the Horizon GUI before you run this cell.

In [2]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node1-project1-{username}", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.medium"
)
s.submit(idempotent=True)

Waiting for server node1-project1-jja435_nyu_edu's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node1-project1-jja435_nyu_edu
Id,93d5014b-710b-45a6-9b22-37d0724b78ef
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.medium
Addresses,sharednet1: IP: 10.56.3.75 (v4) Type: fixed MAC: fa:16:3e:e5:57:2e
Network Name,sharednet1
Created At,2025-04-21T23:43:42Z
Keypair,jja435_nyu_edu-jupyter
Reservation Id,None
Host Id,775275e0ed9ed8d45a315a91910493058aefb260f918f423efb493dd


Then, we’ll associate a floating IP with the instance:

In [3]:
s.associate_floating_ip()

In [4]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.26.161 port 22.


Connection successful


In the output below, make a note of the floating IP that has been assigned to your instance (in the “Addresses” row).

In [5]:
s.refresh()
s.show(type="widget")

Attribute,node1-project1-jja435_nyu_edu
Id,93d5014b-710b-45a6-9b22-37d0724b78ef
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.medium
Addresses,sharednet1: IP: 10.56.3.75 (v4) Type: fixed MAC: fa:16:3e:e5:57:2e IP: 129.114.26.161 (v4) Type: floating MAC: fa:16:3e:e5:57:2e
Network Name,sharednet1
Created At,2025-04-21T23:43:42Z
Keypair,jja435_nyu_edu-jupyter
Reservation Id,None
Host Id,775275e0ed9ed8d45a315a91910493058aefb260f918f423efb493dd


By default, all connections to VM resources are blocked, as a security measure. We need to attach one or more “security groups” to our VM resource, to permit access over the Internet to specified ports.

The following security groups will be created (if they do not already exist in our project) and then added to our server:

In [6]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-5000", 'port': 5000, 'description': "Enable TCP port 5000 (used by Flask)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by FastAPI)"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"},
  {'name': "allow-8080", 'port': 8080, 'description': "Enable TCP port 8080 (used by cAdvisor, Label Studio, Airflow)"},
  {'name': "allow-8081", 'port': 8081, 'description': "Enable TCP port 8081 (alt for 8080)"}

]

In [7]:
# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['allow-5000', 'allow-8000', 'allow-8080', 'allow-8081', 'allow-8888', 'allow-9000', 'allow-9001', 'allow-ssh', 'default']


### Retrieve code and notebooks on the instance

Now, we can use `python-chi` to execute commands on the instance, to set it up. We’ll start by retrieving the code and other materials on the instance.

In [8]:
s.execute("git clone https://github.com/JadeAnt/NYC-Vision-Zero-MLOPS")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.26.161: b'9a86fe116d44cc14ce8a55e39ff0d431'
  warnings.warn(
Cloning into 'NYC-Vision-Zero-MLOPS'...


<Result cmd='git clone https://github.com/JadeAnt/NYC-Vision-Zero-MLOPS' exited=0>

### Set up Docker

Here, we will set up the container framework.

In [9]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null

Running kernel seems to be up-to-date.

Restarting services...
 systemctl restart packagekit.service

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

## Open an SSH session

Finally, open an SSH sesson on your server. From your local terminal, run

    ssh -i ~/.ssh/nyc_vision_zero.pem cc@A.B.C.D

where

-   in place of `~/.ssh/nyc_vision_zero.pem`, substitute the path to your own key that you had uploaded to KVM@TACC
-   in place of `A.B.C.D`, use the floating IP address you just associated to your instance.